In [29]:
from sympy import *
init_printing()

In [30]:
def P_op(In_size, n, m):
    return eye(In_size).elementary_row_op('n<->m', n, m)

def M_op(In_size, n, k):
    return eye(In_size).elementary_row_op('n->kn', n, k)

def A_op(In_size, n, m, k):
    return eye(In_size).elementary_row_op('n->n+km', row1=n, row2=m, k=k)

In [31]:
#A = randMatrix(3, c=5, min=0, max=25,percent=50)
A = Matrix([
      [2, 3, 1],
      [1, 2, 1],
    ])
A

⎡2  3  1⎤
⎢       ⎥
⎣1  2  1⎦

In [32]:
def smart_rref_decomposition(rref):
    elementary_matrices = []
    # количество нулевых строк в rref
    null_row_count = 0
    row = 0
    # вычеркиваем нулевые строки
    while row < rref.shape[0]:
        if rref.row(row) == zeros(1, rref.shape[1]):
            rref.row_del(row)
        row += 1
    # индексы ячеек главной диагонали
    diag_i, diag_j = 0, 0
    while diag_i < rref.shape[0] and diag_j < rref.shape[1]:
        element_to_one = rref[diag_i, diag_j]
        if element_to_one == 0:
            # ищем ненулевой элемент в столбце
            found = False
            for row in range(diag_i + 1, rref.shape[0]):
                # если нашли
                if rref[row, diag_j] != 0:
                    element_to_one = rref[row, diag_j]
                    # перестановка
                    op = P_op(rref.shape[0], diag_i, row)
                    # сохраняем перестановку
                    elementary_matrices.append(op)
                    rref = op * rref
                    found = True
                    break
            # если не нашли
            if found == False:
                # смещаемся вправо
                diag_j += 1
                continue  
        # приводим к единице, если надо
        if element_to_one != 1: 
            op = M_op(rref.shape[0], diag_i, 1 / element_to_one)
            elementary_matrices.append(op)
            rref = op * rref   
        # приводим к нулю элементы под и над данной ячейкой
        row = 0 
        while row < rref.shape[0]:
            if row != diag_i and rref[row, diag_j] != 0 and rref.row(row) != zeros(1, rref.shape[1]):
                element_to_zero = rref[row, diag_j]
                op = A_op(rref.shape[0], row, diag_i, -element_to_zero)
                elementary_matrices.append(op)
                rref = op * rref  
                # если получили нулевую строку,
                # то вычеркиваем ее
                if rref.row(row) == zeros(1, rref.shape[1]):
                    rref.row_del(row)  
                    row -= 1
                    if (row < diag_i):
                        diag_i -= 1
                    # и запоминаем, что вычеркнули
                    null_row_count += 1
            row += 1         
        diag_i += 1
        diag_j += 1            
    # записываем обратно вычеренутые строки
    for i in range(null_row_count):
        rref = rref.row_insert(rref.shape[0], zeros(1, rref.shape[1]))
    return rref, elementary_matrices
rref, elementary_matrices = smart_rref_decomposition(A)
A, A.rref()[0], rref, elementary_matrices

⎛⎡2  3  1⎤  ⎡1  0  -1⎤  ⎡1  0  -1⎤  ⎡⎡1/2  0⎤  ⎡1   0⎤  ⎡1  0⎤  ⎡1  -3/2⎤⎤⎞
⎜⎢       ⎥, ⎢        ⎥, ⎢        ⎥, ⎢⎢      ⎥, ⎢     ⎥, ⎢    ⎥, ⎢       ⎥⎥⎟
⎝⎣1  2  1⎦  ⎣0  1  1 ⎦  ⎣0  1  1 ⎦  ⎣⎣ 0   1⎦  ⎣-1  1⎦  ⎣0  2⎦  ⎣0   1  ⎦⎦⎠

In [33]:
rref2 = A
for e in elementary_matrices:
    rref2 = e * rref2
A,A.rref()[0],rref2

⎛⎡2  3  1⎤  ⎡1  0  -1⎤  ⎡1  0  -1⎤⎞
⎜⎢       ⎥, ⎢        ⎥, ⎢        ⎥⎟
⎝⎣1  2  1⎦  ⎣0  1  1 ⎦  ⎣0  1  1 ⎦⎠

In [34]:
elementary_matrices_reversed = list(reversed(elementary_matrices))
elementary_matrices_reversed

⎡⎡1  -3/2⎤  ⎡1  0⎤  ⎡1   0⎤  ⎡1/2  0⎤⎤
⎢⎢       ⎥, ⎢    ⎥, ⎢     ⎥, ⎢      ⎥⎥
⎣⎣0   1  ⎦  ⎣0  2⎦  ⎣-1  1⎦  ⎣ 0   1⎦⎦

In [35]:
elementary_matrices_reversed_inv = [i.inv() for i in elementary_matrices_reversed]
elementary_matrices_reversed_inv

⎡⎡1  3/2⎤  ⎡1   0 ⎤  ⎡1  0⎤  ⎡2  0⎤⎤
⎢⎢      ⎥, ⎢      ⎥, ⎢    ⎥, ⎢    ⎥⎥
⎣⎣0   1 ⎦  ⎣0  1/2⎦  ⎣1  1⎦  ⎣0  1⎦⎦

In [36]:
A2 = A.rref()[0]
for e_inv in elementary_matrices_reversed_inv:
    A2 = e_inv * A2
A,A2

⎛⎡2  3  1⎤  ⎡2  3  1⎤⎞
⎜⎢       ⎥, ⎢       ⎥⎟
⎝⎣1  2  1⎦  ⎣1  2  1⎦⎠

In [37]:
n = 0
# проверка алгоритма приведения rref на 1000 случайных матриц
for i in range(1000):
    A = randMatrix(4, c=7, min=-30, max=15,percent=50)
    rref, elementary_matrices = smart_rref_decomposition(A)
    # счетчик успешных тестов
    if A.rref()[0] == rref:
        n += 1
    else:
        print("Test failed")
        break
print("Successful tests:", n)

Successful tests: 1000
